In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [4]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

model = keras.Sequential(
    [
        CNNBlock(32),
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/5
938/938 - 201s - loss: 0.5754 - accuracy: 0.9464 - 201s/epoch - 215ms/step
Epoch 2/5
938/938 - 196s - loss: 0.0971 - accuracy: 0.9813 - 196s/epoch - 208ms/step
Epoch 3/5
938/938 - 184s - loss: 0.0346 - accuracy: 0.9893 - 184s/epoch - 197ms/step
Epoch 4/5
938/938 - 147s - loss: 0.0275 - accuracy: 0.9912 - 147s/epoch - 156ms/step
Epoch 5/5
938/938 - 130s - loss: 0.0254 - accuracy: 0.9918 - 130s/epoch - 138ms/step
157/157 - 6s - loss: 0.0567 - accuracy: 0.9846 - 6s/epoch - 36ms/step


[0.05667408928275108, 0.9846000075340271]

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding='same')

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
        return self.pooling(x)

class ResNet_Like(models.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([64, 64, 128])
        self.block3 = ResBlock([128, 128, 256])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        return self.classifier(x)

# Corrected model creation
model = ResNet_Like(num_classes=10)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Assuming you have already loaded your data into x_train, y_train, x_test, y_test
# For example:
# x_train, y_train, x_test, y_test = load_data()  # Replace with your actual data loading code

# Fit the model
model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)

# Print the model summary
model.summary()

# Evaluate the model
model.evaluate(x_test, y_test, batch_size=64, verbose=2)


938/938 - 135s - loss: 0.0868 - accuracy: 0.9738 - 135s/epoch - 144ms/step
Model: "res_net__like_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block_6 (ResBlock)      multiple                  28896     
                                                                 
 res_block_7 (ResBlock)      multiple                  185664    
                                                                 
 res_block_8 (ResBlock)      multiple                  739968    
                                                                 
 global_average_pooling2d_2   multiple                 0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             multiple                  2570      
                                                                 
Total params: 957,098
Trainable params: 95

[0.4082781970500946, 0.8718000054359436]

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding='same')

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
        return self.pooling(x)

class ResNet_Like(models.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([64, 64, 128])
        self.block3 = ResBlock([128, 128, 256])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        return self.classifier(x)

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))

# Corrected model creation
model = ResNet_Like(num_classes=10)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Assuming you have already loaded your data into x_train, y_train, x_test, y_test
# For example:
# x_train, y_train, x_test, y_test = load_data()  # Replace with your actual data loading code

# Fit the model
model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)

# Print the model summary
model.model().summary()

# Evaluate the model
model.evaluate(x_test, y_test, batch_size=64, verbose=2)


938/938 - 123s - loss: 0.0853 - accuracy: 0.9747 - 123s/epoch - 131ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block_9 (ResBlock)      (None, 14, 14, 64)        28896     
                                                                 
 res_block_10 (ResBlock)     (None, 7, 7, 128)         185664    
                                                                 
 res_block_11 (ResBlock)     (None, 3, 3, 256)         739968    
                                                                 
 global_average_pooling2d_3   (None, 256)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 10)             

[0.03715193271636963, 0.9883000254631042]